In [ ]:
import tensorflow as tf 
from tensorflow.keras.layers import Flatten, Dense, Dropout
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Loading the Xception model
base_model = tf.keras.applications.Xception(
    include_top=False,
    weights="xception_weights_tf_dim_ordering_tf_kernels_notop.h5",
    input_shape=(256, 256, 3),
)

In [ ]:
# Summary of the model
base_model.summary()

In [ ]:
# Making layers of base model untrainable so that their weights won't change during training for first few epochs
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Defining the input and output layers of the base model
base_input = base_model.layers[0].input
base_output = base_model.layers[-1].output

In [ ]:
# Adding the output layers to the base model
l1 = Flatten()(base_output)
l2 = Dense(512, activation='elu')(l1)
l3 = Dropout(0.2)(l2)
l4 = Dense(256, activation='elu')(l3)
l5 = Dropout(0.2)(l4)
l6 = Dense(12, activation='softmax')(l5)

model = tf.keras.Model(inputs=base_input, outputs=l6)

In [ ]:
#Importing the data and augmenting it:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.15,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        'C://Users//mukul//Documents//#Programing//Skin_Pigment_Analysis//Dataset//OG_Train',
        target_size=(256, 256),
        batch_size=69,
        class_mode='categorical')


valid_datagen = ImageDataGenerator(rescale=1/255)

valid_generator = valid_datagen.flow_from_directory(
        'C://Users//mukul//Documents//#Programing//Skin_Pigment_Analysis//Dataset//OG_Valid',
        target_size=(256, 256),
        batch_size=17,
        class_mode='categorical')

In [ ]:
# Compiling the model i.e selecting the optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
model.compile(optimizer=optimizer, loss = 'categorical_crossentropy', metrics = ['acc'])

In [ ]:
# training the model for few epoch so that the weights of pretrained layers don't change abruptly
history = model.fit(
      train_generator,
      steps_per_epoch=25,
      validation_data=(valid_generator),  
      epochs=5,
      verbose=1)

In [ ]:
# Making some layers of base model trainable 
for layer in base_model.layers[-6:-1]:
    layer.trainable = True

In [ ]:
# Setting up the callbacks

# checkpoint_cb will save the model which will perform the best on validation set
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('C://Users//mukul//Documents//#Programing//Skin_Pigment_Analysis//best_model.h5', save_best_only=True)

# early_stoping_cb will stop the model traning once it stops improving
early_stoping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [ ]:
# Compiling the model i.e selecting the optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)
model.compile(optimizer=optimizer, loss = 'categorical_crossentropy', metrics = ['acc'])

In [ ]:
# training the model
history = model.fit(
      train_generator,
      steps_per_epoch=,
      validation_data=(valid_generator),
      callbacks=[checkpoint_cb, early_stoping_cb],  
      epochs=100,
      verbose=1)

In [ ]:
# Performance of the model on training and validation sets
pd.DataFrame(history.history).plot()
plt.gca().set_ylim(0, 1)
plt.show()